In [321]:
import pandas as pd
import pandas_datareader as pdr
import datetime
import yfinance as yf
import numpy as np
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import matplotlib.pyplot as plt
import xlrd
from featexp import get_univariate_plots

pd.options.mode.chained_assignment = None
np.set_printoptions(suppress=True)

#FEATURES
#Prediction? Number of days trending upwards? Percentage increase?
#Sell after a 5% increase?
#Dissect what all of this means.
#I want to get a precision / recall curve
#Set rounding threshold lower
#Confidence variables?
#Try other models?
#Try other stocks besides stocks that tend to just go up?
#Predict rise in values rather than just simple classification?
#Set rise rate higher / faster?
#Invest money based on confidence / probability?
#We call more about precision & accuracy than recall
#Remove unnecessary features, prevent overfitting?
#Paper trade over previous data...see how we would have fared?
#Automate all of this?
#NaN values a problem?
#Overfitting based on general bullish tech stocks?
#Rise 5% within 10 days or AT 10 days?
#get model size / kernel size


#TODOS
#check calculations. RSI in particular. 9 or 10 days?
#Understand features, overfitting. 
#Normalize MACD?
#Why is RSI 5 sometimes NaN? That can't be right
#Adjust strategy as appropriate
#More features needed?
#Change target to go up by 5% WITHIN 10 days not AT 10 days

In [322]:
def RSI_calc(start, lookback, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]]
        
        df['GAIN_LOSS'] = df.apply(lambda row : (row["CLOSE"]-row["OPEN"]) / row["OPEN"] * 100, axis=1)
        avg_gain = df.loc[df['GAIN_LOSS'] >= 0]["GAIN_LOSS"].mean()
        avg_loss = df.loc[df['GAIN_LOSS'] < 0]["GAIN_LOSS"].mean() * -1
        RSI = 100 - (100 / (1 + (avg_gain/avg_loss)))
        return RSI

In [323]:
def daily_return(start, lookback, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1] #Include day?
        df['daily_change'] = df.apply(lambda row: ((row['CLOSE'] - row['OPEN']) / row['OPEN']) * 100, axis=1)
        avg_daily_return = df['daily_change'].mean()
        return avg_daily_return

In [324]:
def weekly_return(start, lookback, df, date_time=True): #Check math here!!
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1] #Include day?
        df = df.iloc[::5, :]
        df = df.reset_index()
        weekly_change_sum = 0
        for i in range(1, len(df)): #Would really like to not be hacky about this...but oh well
            weekly_change_sum += ((df.iloc[i]['CLOSE'] - df.iloc[i-1]['CLOSE']) / df.iloc[i-1]['OPEN']) * 100
        weekly_change_avg = weekly_change_sum / len(df)-1
        return weekly_change_avg

In [325]:
def monthly_return(start, lookback, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        if(lookback < 30):
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1] #Include day?
        df = df.iloc[::30, :]
        df = df.reset_index()
        monthly_change_sum = 0
        for i in range(1, len(df)): #Would really like to not be hacky about this...but oh well
            monthly_change_sum += ((df.iloc[i]['CLOSE'] - df.iloc[i-1]['CLOSE']) / df.iloc[i-1]['OPEN']) * 100
        monthly_change_avg = monthly_change_sum / len(df)-1
        return monthly_change_avg

In [326]:
def EMA(start, lookback, df):
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1]
        df = df.reset_index()
        SMA = (df['CLOSE'][0:lookback-1].sum()) / lookback
        k = 2 / (lookback + 1)
        EMA = (df.loc[df['DATE']==start]['CLOSE'] * k) + (SMA * (1-k))
        return EMA
    
def MACD(start, lookback1, lookback2, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback2) < 0:
            return None
        EMA_short = EMA(start, lookback1, df)
        EMA_long = EMA(start, lookback2, df)
        return float(EMA_short) - float(EMA_long)

In [327]:
def ratio_avg_to_close(start, lookback, df, date_time=True): #Check math here!!
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1]
        df = df.reset_index()
        mean_close = df[0:lookback]['CLOSE'].sum() / lookback
        ratio = mean_close / df.iloc[lookback]['CLOSE']
        return ratio

In [328]:
def target(start, look_forward, df, target, date_time=True): #Within 10 days or AT 10 days? Also check 9 or 10 days
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])+look_forward) > len(df):
            return None
        df = df[(df.index[df["DATE"]==start][0]):df.index[df["DATE"]==start][0]+look_forward] #+1? 9 or 10 days?
        df = df.reset_index()
        percent_increase = (df.iloc[look_forward-1]['CLOSE'] - df.iloc[0]['CLOSE']) / df.iloc[0]['CLOSE'] * 100
        if percent_increase >= target:
            return True
        else:
            return False

In [329]:
def calcutron(stock_data, stock_symbols, csv_export, csv_name, percent_increase, days_increase):
    df_list = []
    for stock in stock_symbols:
        df = stock_data.loc[stock_data["TICKER"]==stock]
        for num in [5,10,30,60]:
            df['RSI ' + str(num)] = df.apply(lambda row: RSI_calc(row['DATE'], num, df, True), axis=1)
            df["AVG DAILY RETURN " + str(num)] = df.apply(lambda row: daily_return(row['DATE'], num, df, True), axis=1)
            df["AVG WEEKLY RETURN " + str(num)] = df.apply(lambda row: weekly_return(row['DATE'], num, df, True), axis=1)
            df["AVG MONTHLY RETURN " + str(num)] = df.apply(lambda row: monthly_return(row['DATE'], num, df, True), axis=1)
            df["RATIO " + str(num)] = df.apply(lambda row: ratio_avg_to_close(row['DATE'], num, df, True), axis=1)
        df["MACD 10_30"] = df.apply(lambda row: MACD(row['DATE'], 10, 30, df, True), axis=1)
        df["MACD 5_10"] = df.apply(lambda row: MACD(row['DATE'], 5, 10, df, True), axis=1)
        df["MACD 2_10"] = df.apply(lambda row: MACD(row['DATE'], 2, 10, df, True), axis=1)
        df['MACD 10_30 DIFF'] = df['MACD 10_30'].diff()
        df['MACD 5_10 DIFF'] = df['MACD 5_10'].diff()
        df['MACD 2_10 DIFF'] = df['MACD 2_10'].diff()
        df["TARGET"] = df.apply(lambda row: target(row['DATE'], days_increase, df, percent_increase, True), axis=1)
        df_list.append(df)
    stock_data = pd.concat(df_list)
    stock_data['TARGET'].value_counts(normalize=True) * 100
    if csv_export:
        stock_data.to_excel(csv_name)
    return stock_data

In [352]:
def model_creation(train_stocks, test_size, read_csv, export_csv, csv_name, days, percent_increase, days_increase, features=None, prints=False):
    if read_csv:
        stock_data = pd.read_excel(csv_name, engine='openpyxl')
    else:
        stock_data = pd.DataFrame(columns=["Ticker","Date","Open","High","Low","Close"])
        for stock in train_stocks:
            y_finance_data = yf.Ticker(stock).history(start=(datetime.date.today()-datetime.timedelta(days=days)).strftime("%Y-%m-%d")).reset_index()
            y_finance_data.insert(0,"Ticker", stock)
            y_finance_data = y_finance_data[['Ticker', 'Date','Open','High','Low','Close']]
            stock_data = pd.concat([stock_data, y_finance_data])
        stock_data = stock_data.rename(columns={"Ticker":"TICKER", "Date":"DATE", "Open":"OPEN", "High":"HIGH", "Low":"LOW", "Close":"CLOSE"})
        stock_data = calcutron(stock_data, train_stocks, export_csv, csv_name, percent_increase, days_increase)

    stock_data = stock_data.drop(['AVG MONTHLY RETURN 5','AVG MONTHLY RETURN 10'], axis=1)
    stock_data = stock_data.dropna()
    print(stock_data)
    if features == None:
        input_patterns = stock_data.loc[:,'RSI 5':'MACD 2_10 DIFF'][60:len(stock_data)-10]
    else:
        input_patterns = stock_data[features][60:len(stock_data)-10]
    
    input_label = stock_data['TARGET'][60:len(stock_data)-10].astype(int) #Why are these floats? It bothers me.
    if prints:
        print(input_patterns)
        print(stock_data['TARGET'].value_counts(normalize=True) * 100)
    
    accuracy_trials = []
    recall_trials = []
    precision_trials = []

    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(input_patterns, input_label, test_size = test_size)
        model = XGBClassifier(eval_metric="logloss", use_label_encoder=False)
        model.fit(X_train, y_train)
        accuracy_trials.append(accuracy_score(y_test, model.predict(X_test)) * 100.0)
        precision_trials.append(precision_score(y_test, model.predict(X_test)) * 100.0)
        recall_trials.append(recall_score(y_test, model.predict(X_test)) * 100.0)
        average_precision_trials.append(average_precision_score(y_test, model.predict(X_test)))

    mean_accuracy = sum(accuracy_trials) / len(accuracy_trials)
    mean_precision = sum(precision_trials) / len(precision_trials)
    mean_recall= sum(recall_trials) / len(recall_trials)

    if prints:
        print("Validation Trial")
        print("Mean Accuracy: {}%".format(mean_accuracy))
        print("Mean Precision: {}%".format(mean_precision))
        print("Mean Recall: {}%".format(mean_recall))
        print(model)
    return model, stock_data, X_train, X_test, y_train, y_test

In [331]:
def trials(model, threshold, test_stocks, days, export_csv, csv_name, percent_increase, days_increase, features, prints=False):
    #Add ability to read from CSV here?
    for stock in test_stocks:
        print(stock)
        stock_data_testing = pd.DataFrame(columns=["Ticker","Date","Open","High","Low","Close"])
        y_finance_data = yf.Ticker(stock).history(start=(datetime.date.today()-datetime.timedelta(days=days)).strftime("%Y-%m-%d")).reset_index()
        y_finance_data.insert(0,"Ticker", stock)
        y_finance_data = y_finance_data[['Ticker', 'Date','Open','High','Low','Close']]

        stock_data_testing = pd.concat([stock_data_testing, y_finance_data])
        stock_data_testing = stock_data_testing.rename(columns={"Ticker":"TICKER", "Date":"DATE", "Open":"OPEN", "High":"HIGH", "Low":"LOW", "Close":"CLOSE"})
        stock_data_testing = calcutron(stock_data_testing, [stock], False, None, percent_increase, days_increase)
        stock_data_testing = stock_data_testing.drop(['AVG MONTHLY RETURN 5','AVG MONTHLY RETURN 10'], axis=1)
        stock_data_testing = stock_data_testing.dropna()
        if features == None:
            input_patterns_testing = stock_data_testing.loc[:,'RSI 5':'MACD 2_10 DIFF'][60:len(stock_data)-10]
        else:
            input_patterns_testing = stock_data_testing[features][60:len(stock_data)-10]
    
        input_label_testing = stock_data_testing['TARGET'][60:len(stock_data)-10].astype(int)
        print(input_patterns_testing)
        #if prints:
            #print(stock_data_testing)
            #print(input_patterns_testing)
            #print(input_label_testing)

        accuracy_trials_testing = []
        recall_trials_testing = []
        precision_trials_testing = []

        y_prob = model.predict_proba(input_patterns_testing)
        y_prob_round = []
        for row in y_prob:
            if row[1] > threshold:
                y_prob_round.append(1)
            else:
                y_prob_round.append(0)
        y_pred = model.predict(input_patterns_testing)
        
        accuracy_trials_testing.append(accuracy_score(input_label_testing, y_pred) * 100.0)
        precision_trials_testing.append(precision_score(input_label_testing, y_pred) * 100.0)
        recall_trials_testing.append(recall_score(input_label_testing, y_pred) * 100.0)
        mean_accuracy_testing = sum(accuracy_trials_testing) / len(accuracy_trials_testing)
        mean_precision_testing = sum(precision_trials_testing) / len(precision_trials_testing)
        mean_recall_testing = sum(recall_trials_testing) / len(recall_trials_testing)

        if prints:
            print(stock)
            print("Standard Threshold")
            print("Mean Accuracy {}: {}%".format(stock, mean_accuracy_testing))
            print("Mean Precision {}: {}%".format(stock, mean_precision_testing))
            print("Mean Recall {}: {}%".format(stock, mean_recall_testing))
            print()

        accuracy_trials_testing.append(accuracy_score(input_label_testing, y_prob_round) * 100.0)
        precision_trials_testing.append(precision_score(input_label_testing, y_prob_round) * 100.0)
        recall_trials_testing.append(recall_score(input_label_testing, y_prob_round) * 100.0)
        mean_accuracy_testing = sum(accuracy_trials_testing) / len(accuracy_trials_testing)
        mean_precision_testing = sum(precision_trials_testing) / len(precision_trials_testing)
        mean_recall_testing = sum(recall_trials_testing) / len(recall_trials_testing)

        if prints:
            print("Rounding Threshold: {}".format(threshold))
            print("Mean Accuracy {}: {}%".format(stock, mean_accuracy_testing))
            print("Mean Precision {}: {}%".format(stock, mean_precision_testing))
            print("Mean Recall {}: {}%".format(stock, mean_recall_testing))
            
    return stock_data

In [ ]:
stock_symbols = ["KR", "WMT", "CPB", "KO", "NVDA", "BAC", "F", "XOM", "GOOG", "AMD"]
#features = ["RSI 10", "RSI 30", "RSI 60", "AVG DAILY RETURN 5", "AVG DAILY RETURN 10", "AVG DAILY RETURN 30", "AVG DAILY RETURN 60", "AVG WEEKLY RETURN 5", "AVG WEEKLY RETURN 10", "AVG WEEKLY RETURN 30", "AVG WEEKLY RETURN 60", "RATIO 5", "RATIO 10", "RATIO 30", "RATIO 60", "AVG MONTHLY RETURN 30", "AVG MONTHLY RETURN 60", "MACD 10_30 DIFF", "MACD 5_10 DIFF", "MACD 2_10 DIFF"]
model, stock_data, X_train, X_test, y_train, y_test = model_creation(stock_symbols, .33, False, True, "MAJOR_STONK.xlsx", 3600, 5, 10, features=None, prints=True)

     TICKER       DATE       OPEN       HIGH        LOW      CLOSE      RSI 5  \
60       KR 2011-08-10   9.382187   9.474622   9.184712   9.214123  13.338367   
61       KR 2011-08-11   9.237343   9.646861   9.190904   9.545537   3.590095   
62       KR 2011-08-12   9.634194   9.689078   9.418881   9.596197  39.122200   
63       KR 2011-08-15   9.613087   9.684858   9.537094   9.655305  45.200551   
64       KR 2011-08-16   9.604641   9.832620   9.575089   9.743961  54.100153   
...     ...        ...        ...        ...        ...        ...        ...   
2464    AMD 2021-03-02  86.919998  86.949997  84.040001  84.129997  28.741948   
2465    AMD 2021-03-03  84.279999  84.379997  80.849998  80.860001  32.312095   
2466    AMD 2021-03-04  80.230003  81.809998  76.779999  77.750000  23.043919   
2467    AMD 2021-03-05  79.000000  79.480003  74.199997  78.519997  25.143352   
2468    AMD 2021-03-08  78.029999  79.000000  73.860001  73.959999  30.352394   

      AVG DAILY RETURN 5  A

In [346]:
test_stocks = ["AAPL", "X", "TGT", "AMZN", "INTC", "JPM", "KO", "NVDA"] 
stock_data_test = trials(model, .85, test_stocks, 1800, False, None, 5, 10, features=None, prints=True)

AAPL


KeyboardInterrupt: 

In [ ]:
X_train['TARGET'] = y_train
X_test['TARGET'] = y_test
get_univariate_plots(data=X_train, target_col ="TARGET", data_test=X_test)

In [ ]:
from joblib import dump, load
dump(model, 'trained_model_all_features.joblib') 